In [1]:
using LinearAlgebra
using Printf

function gauss_solve(M::Matrix{Float64}, b::Vector{Float64})
    n = size(M, 1)
    A = hcat(M, b)

    for k in 1:n-1
        pivot = argmax(abs.(A[k:end, k])) + k - 1
        if pivot != k
            A[[k, pivot], :] .= A[[pivot, k], :]
        end

        for i in k+1:n
            factor = A[i, k] / A[k, k]
            A[i, k:end] .-= factor .* A[k, k:end]
        end
    end

    x = zeros(Float64, n)
    for i in n:-1:1
        x[i] = (A[i, end] - sum(A[i, i+1:n] .* x[i+1:n])) / A[i, i]
    end

    return x
end


function krylov_eigenvalues(A::Matrix{Float64})
    n = size(A, 1)
    y0 = zeros(n)
    y0[1] = 1.0

    Y = zeros(n, n + 1)
    Y[:, 1] = y0
    for k in 2:n+1
        Y[:, k] = A * Y[:, k-1]
    end

    M = zeros(n, n)
    for i in 1:n
        M[:, i] = Y[:, n - i + 1]
    end
    b = Y[:, n + 1]

    p = gauss_solve(M, b)

    F = zeros(n, n)
    F[1, :] = p
    for i in 2:n
        F[i, i-1] = 1.0
    end
    return eigvals(F), Y, p
end


function eigenvectors_krylov(A, eigenvalues, Y, p)
    n = length(eigenvalues)
    eigenvectors = Vector{Vector{ComplexF64}}()

    for λ in eigenvalues
        q = zeros(ComplexF64, n)
        q[1] = 1.0
        for j in 2:n
            q[j] = λ * q[j-1]
            if j <= n
                q[j] -= p[j-1]
            end
        end

        x = zeros(ComplexF64, n)
        for j in 1:n
            x .+= q[j] .* Y[:, n - j + 1]
        end

        if norm(x) > 0
            x ./= norm(x)
        end

        push!(eigenvectors, x)
    end

    return eigenvectors
end


function method_krylov(A::Matrix{Float64})
    eigenvalues, Y, p = krylov_eigenvalues(A)
    eigenvectors = eigenvectors_krylov(A, eigenvalues, Y, p)
    return eigenvalues, hcat(eigenvectors...)
end


real_tol(x; tol=1e-10) = x isa Complex ? (abs(imag(x)) ≤ tol ? real(x) : x) : x

function make_real_eigs(λs, X; tol=1e-10)
    λr = [real_tol(λ; tol=tol) for λ in λs]
    Y = similar(X)
    for j in 1:size(X,2)
        v = X[:,j]
        if maximum(abs.(imag.(ComplexF64.(v)))) ≤ tol
            w = real.(v)
            k = argmax(abs.(w))
            w = w[k] < 0 ? -w : w
            Y[:,j] = w
        else
            Y[:,j] = v
        end
    end
    λr, Y
end

function print_eigs_col(λs, X; tol=1e-10, digits=6)
    println("Собственные значения матрицы:")
    for (j,λ) in enumerate(λs)
        @printf("λ%-2d = %.*f\n", j, digits, real_tol(λ; tol=tol))
    end
    println("\nСобственные векторы матрицы:")
    for j in 1:size(X,2)
        println("x$j =")
        for i in 1:size(X,1)
            x = real_tol(X[i,j]; tol=tol)
            if x isa Complex
                @printf("  %.*f %+.*fi\n", digits, real(x), digits, imag(x))
            else
                @printf("  %.*f\n", digits, x)
            end
        end
        println()
    end
end


A = [2.2  1.0  0.5  2.0;
     1.0  1.3  2.0  1.0;
     0.5  2.0  0.5  1.6;
     2.0  1.0  1.6  2.0]

λs, X = method_krylov(A)
λs, X = make_real_eigs(λs, X; tol=1e-10)
print_eigs_col(λs, X; tol=1e-10, digits=6)


Собственные значения матрицы:
λ1  = -1.420087
λ2  = 0.222636
λ3  = 1.545418
λ4  = 5.652032

Собственные векторы матрицы:
x1 =
  0.222043
  -0.515910
  0.757274
  -0.333271

x2 =
  -0.521921
  -0.454869
  0.153447
  0.705086

x3 =
  0.628930
  -0.572574
  -0.485654
  0.201858

x4 =
  0.531736
  0.446194
  0.408816
  0.592484

